In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-credit-default-risk/sample_submission.csv
/kaggle/input/home-credit-default-risk/bureau_balance.csv
/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv
/kaggle/input/home-credit-default-risk/application_train.csv
/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv
/kaggle/input/home-credit-default-risk/application_test.csv
/kaggle/input/home-credit-default-risk/previous_application.csv
/kaggle/input/home-credit-default-risk/credit_card_balance.csv
/kaggle/input/home-credit-default-risk/installments_payments.csv
/kaggle/input/home-credit-default-risk/bureau.csv


In [2]:
df_previous = pd.read_csv('/kaggle/input/home-credit-default-risk/previous_application.csv')
df_cash = pd.read_csv('/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv')
df_instalments = pd.read_csv('/kaggle/input/home-credit-default-risk/installments_payments.csv')
df_credit_card = pd.read_csv('/kaggle/input/home-credit-default-risk/credit_card_balance.csv')
print(df_previous.shape)
print(df_cash.shape)
print(df_instalments.shape)
print(df_credit_card.shape)


(1670214, 37)
(10001358, 8)
(13605401, 8)
(3840312, 23)


## Encoding and imputing previous_application.csv 
### Imputing missing values since there were no string values to encode

In [3]:
previous_categorical_columns = df_previous.select_dtypes(exclude='number').columns
df_previous_categorical = df_previous[previous_categorical_columns].copy()
column_values_previous = df_previous_categorical.nunique()

In [4]:
label_previous=[]
one_hot_previous=[]

for i in range(0,len(column_values_previous)):
    if column_values_previous.iloc[i]>2:
        one_hot_previous.append(column_values_previous.index[i])
    else:
        label_previous.append(column_values_previous.index[i])

print(label_previous)
print(one_hot_previous)

['FLAG_LAST_APPL_PER_CONTRACT']
['NAME_CONTRACT_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_CASH_LOAN_PURPOSE', 'NAME_CONTRACT_STATUS', 'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON', 'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE', 'NAME_SELLER_INDUSTRY', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION']


In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

hot_encoder_previous = OneHotEncoder(handle_unknown = 'ignore',sparse_output=False)
ordinal_encoder_previous = OrdinalEncoder(handle_unknown = 'use_encoded_value',unknown_value = -1)

hot_previous_features = pd.DataFrame(hot_encoder_previous.fit_transform(df_previous_categorical[one_hot_previous]),columns=hot_encoder_previous.get_feature_names_out(one_hot_previous))
ordinal_previous_features = pd.DataFrame(ordinal_encoder_previous.fit_transform(df_previous_categorical[label_previous]),columns=label_previous)

df_previous_categorical = pd.concat([hot_previous_features,ordinal_previous_features],axis=1)
df_previous_categorical

,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Consumer loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_XNA,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,...,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest,PRODUCT_COMBINATION_nan,FLAG_LAST_APPL_PER_CONTRACT
0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670209,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1670210,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1670211,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1670212,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [6]:
previous_numerical_columns = df_previous.select_dtypes(include = 'number').columns
df_previous_numerical = df_previous[previous_numerical_columns].copy()

df_previous = pd.concat([df_previous_numerical,df_previous_categorical],axis=1)
df_previous

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,...,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest,PRODUCT_COMBINATION_nan,FLAG_LAST_APPL_PER_CONTRACT
0,2030495,271877,1730.430,17145.0,17145.0,0.0,17145.0,15,1,0.000000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,2802425,108129,25188.615,607500.0,679671.0,NaN,607500.0,11,1,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2523466,122040,15060.735,112500.0,136444.5,NaN,112500.0,11,1,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2819243,176158,47041.335,450000.0,470790.0,NaN,450000.0,7,1,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1784265,202054,31924.395,337500.0,404055.0,NaN,337500.0,9,1,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670209,2300464,352015,14704.290,267295.5,311400.0,0.0,267295.5,12,1,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1670210,2357031,334635,6622.020,87750.0,64291.5,29250.0,87750.0,15,1,0.340554,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1670211,2659632,249544,11520.855,105237.0,102523.5,10525.5,105237.0,12,1,0.101401,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1670212,2785582,400317,18821.520,180000.0,191880.0,NaN,180000.0,9,1,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [7]:
empty_columns=[]

for column in df_previous.columns:
    if df_previous[column].isnull().sum()/1670214>0.9:
        empty_columns.append(column)

df_previous= df_previous.drop(columns=empty_columns,axis=1)

print(empty_columns)
print(len(empty_columns))


['RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED']
2


In [8]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy = 'constant',fill_value = -1)
df_previous = pd.DataFrame(imputer.fit_transform(df_previous),columns=df_previous.columns)
df_previous

,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,...,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest,PRODUCT_COMBINATION_nan,FLAG_LAST_APPL_PER_CONTRACT
0,2030495.0,271877.0,1730.430,17145.0,17145.0,0.0,17145.0,15.0,1.0,0.000000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,2802425.0,108129.0,25188.615,607500.0,679671.0,-1.0,607500.0,11.0,1.0,-1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2523466.0,122040.0,15060.735,112500.0,136444.5,-1.0,112500.0,11.0,1.0,-1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2819243.0,176158.0,47041.335,450000.0,470790.0,-1.0,450000.0,7.0,1.0,-1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1784265.0,202054.0,31924.395,337500.0,404055.0,-1.0,337500.0,9.0,1.0,-1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670209,2300464.0,352015.0,14704.290,267295.5,311400.0,0.0,267295.5,12.0,1.0,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1670210,2357031.0,334635.0,6622.020,87750.0,64291.5,29250.0,87750.0,15.0,1.0,0.340554,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1670211,2659632.0,249544.0,11520.855,105237.0,102523.5,10525.5,105237.0,12.0,1.0,0.101401,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1670212,2785582.0,400317.0,18821.520,180000.0,191880.0,-1.0,180000.0,9.0,1.0,-1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
# Manual feature engineering based on columns

df_previous['PREVIOUS_MONHTLY'] = df_previous['AMT_ANNUITY']/12
df_previous['PREVIOUS_CREDIT_DELTA'] = df_previous.apply(lambda row: (row['AMT_CREDIT']- row['AMT_APPLICATION']) if (row['AMT_CREDIT'] != -1 and row['AMT_APPLICATION']!=-1) else -1,axis=1)
df_previous['PREVIOUS_GOODS_DELTA'] = df_previous.apply(lambda row : (row['AMT_CREDIT']-row['AMT_GOODS_PRICE']) if (row['AMT_CREDIT'] != -1 and row['AMT_GOODS_PRICE']!=-1) else -1,axis=1)
df_previous['PREVIOUS_LOANS_CNT'] = df_previous.groupby(['SK_ID_CURR'])['SK_ID_CURR'].transform('count')

df_previous


,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,...,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest,PRODUCT_COMBINATION_nan,FLAG_LAST_APPL_PER_CONTRACT,PREVIOUS_MONHTLY,PREVIOUS_CREDIT_DELTA,PREVIOUS_GOODS_DELTA,PREVIOUS_LOANS_CNT
0,2030495.0,271877.0,1730.430,17145.0,17145.0,0.0,17145.0,15.0,1.0,0.000000,...,1.0,0.0,0.0,0.0,0.0,1.0,144.20250,0.0,0.0,3
1,2802425.0,108129.0,25188.615,607500.0,679671.0,-1.0,607500.0,11.0,1.0,-1.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,2099.05125,72171.0,72171.0,6
2,2523466.0,122040.0,15060.735,112500.0,136444.5,-1.0,112500.0,11.0,1.0,-1.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,1255.06125,23944.5,23944.5,4
3,2819243.0,176158.0,47041.335,450000.0,470790.0,-1.0,450000.0,7.0,1.0,-1.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,3920.11125,20790.0,20790.0,23
4,1784265.0,202054.0,31924.395,337500.0,404055.0,-1.0,337500.0,9.0,1.0,-1.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,2660.36625,66555.0,66555.0,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670209,2300464.0,352015.0,14704.290,267295.5,311400.0,0.0,267295.5,12.0,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,1225.35750,44104.5,44104.5,5
1670210,2357031.0,334635.0,6622.020,87750.0,64291.5,29250.0,87750.0,15.0,1.0,0.340554,...,0.0,0.0,0.0,0.0,0.0,1.0,551.83500,-23458.5,-23458.5,7
1670211,2659632.0,249544.0,11520.855,105237.0,102523.5,10525.5,105237.0,12.0,1.0,0.101401,...,0.0,0.0,0.0,0.0,0.0,1.0,960.07125,-2713.5,-2713.5,3
1670212,2785582.0,400317.0,18821.520,180000.0,191880.0,-1.0,180000.0,9.0,1.0,-1.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,1568.46000,11880.0,11880.0,9


In [10]:
temp = df_previous.copy()
temp = temp.drop('SK_ID_PREV',axis=1)
temp_aggregated = temp.groupby(['SK_ID_CURR'],as_index=False).agg(['mean','sum','max','min'])


temp_aggregated.head()

SK_ID_CURR AMT_ANNUITY                                  AMT_APPLICATION  \
                    mean         sum        max       min            mean   
0   100001.0    3951.000    3951.000   3951.000  3951.000        24835.50   
1   100002.0    9251.775    9251.775   9251.775  9251.775       179055.00   
2   100003.0   56553.990  169661.970  98356.995  6737.310       435436.50   
3   100004.0    5357.250    5357.250   5357.250  5357.250        24282.00   
4   100005.0    2406.100    4812.200   4813.200    -1.000        22308.75   

                                 AMT_CREDIT  ... PREVIOUS_CREDIT_DELTA  \
         sum       max       min       mean  ...                   max   
0    24835.5   24835.5   24835.5   23787.00  ...               -1048.5   
1   179055.0  179055.0  179055.0  179055.00  ...                   0.0   
2  1306309.5  900000.0   68809.5  484191.00  ...              135882.0   
3    24282.0   24282.0   24282.0   20106.00  ...               -4176.0   
4    44617.5   44617.5       0.0   20076.75  ...                   0.0   

          PREVIOUS_GOODS_DELTA                             PREVIOUS_LOANS_CNT  \
      min                 mean       sum       max     min               mean   
0 -1048.5              -1048.5   -1048.5   -1048.5 -1048.5                1.0   
1     0.0                  0.0       0.0       0.0     0.0                1.0   
2  -756.0              48754.5  146263.5  135882.0  -756.0                3.0   
3 -4176.0              -4176.0   -4176.0   -4176.0 -4176.0                1.0   
4 -4464.0              -2232.5   -4465.0      -1.0 -4464.0                2.0   

               
  sum max min  
0   1   1   1  
1   1   1   1  
2   9   3   3  
3   1   1   1  
4   4   2   2  

[5 rows x 661 columns]

In [11]:
temp_aggregated.columns = ['_'.join(col).strip('_') for col in temp_aggregated.columns.values]

In [12]:
temp_aggregated.shape

(338857, 661)

In [13]:
df_previous.isnull().sum()

SK_ID_PREV                     0
SK_ID_CURR                     0
AMT_ANNUITY                    0
AMT_APPLICATION                0
AMT_CREDIT                     0
                              ..
FLAG_LAST_APPL_PER_CONTRACT    0
PREVIOUS_MONHTLY               0
PREVIOUS_CREDIT_DELTA          0
PREVIOUS_GOODS_DELTA           0
PREVIOUS_LOANS_CNT             0
Length: 167, dtype: int64

# Encoding and imputing POS_CASH
## Encoding

In [14]:
columns_cash_categorical = df_cash.select_dtypes(exclude='number').columns
df_cash_categorical = df_cash[columns_cash_categorical].copy()
column_values_cash = df_cash_categorical.nunique()

In [15]:
label_cash=[]
one_hot_cash=[]

for i in range(0,len(column_values_cash)):
    if column_values_previous.iloc[i]>2:
        one_hot_cash.append(column_values_cash.index[i])
    else:
        label_cash.append(column_values_cash.index[i])

print(label_cash)
print(one_hot_cash)

[]
['NAME_CONTRACT_STATUS']


In [16]:
from sklearn.preprocessing import OneHotEncoder

hot_encoder_cash = OneHotEncoder(handle_unknown = 'ignore',sparse_output=False)

hot_cash_features = pd.DataFrame(hot_encoder_cash.fit_transform(df_cash_categorical[one_hot_cash]),columns=hot_encoder_cash.get_feature_names_out(one_hot_cash))



In [17]:
cash_numerical_columns = df_cash.select_dtypes(include = 'number').columns
df_cash_numerical = df_cash[cash_numerical_columns].copy()

df_cash = pd.concat([df_cash_numerical,hot_cash_features],axis=1)
df_cash

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Amortized debt,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_XNA
0,1803195,182943,-31,48.0,45.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1715348,367990,-33,36.0,35.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1784872,397406,-32,12.0,9.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1903291,269225,-35,48.0,42.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2341044,334279,-35,36.0,35.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10001353,2448283,226558,-20,6.0,0.0,843,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10001354,1717234,141565,-19,12.0,0.0,602,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10001355,1283126,315695,-21,10.0,0.0,609,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10001356,1082516,450255,-22,12.0,0.0,614,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Imputing

In [18]:
empty_columns = []

for col in df_cash.columns:
    if (df_cash[col].isnull().sum()/10001358 > 0.9):
        empty_columns.append(col)

df_cash = df_cash.drop(columns=empty_columns,axis=1)
print(len(empty_columns))
df_cash.head()

0


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Amortized debt,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_XNA
0,1803195,182943,-31,48.0,45.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1715348,367990,-33,36.0,35.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1784872,397406,-32,12.0,9.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1903291,269225,-35,48.0,42.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2341044,334279,-35,36.0,35.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer(strategy='constant',fill_value=-1,missing_values = np.nan)
df_cash = pd.DataFrame(imp_mean.fit_transform(df_cash),columns=df_cash.columns)
df_cash.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Amortized debt,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_XNA
0,1803195.0,182943.0,-31.0,48.0,45.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1715348.0,367990.0,-33.0,36.0,35.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1784872.0,397406.0,-32.0,12.0,9.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1903291.0,269225.0,-35.0,48.0,42.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2341044.0,334279.0,-35.0,36.0,35.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
df_cash = df_cash.drop('SK_ID_PREV',axis=1)


In [21]:
aggregated_cash = df_cash.groupby(['SK_ID_CURR'],as_index=False).agg(['mean','sum','max','min'])

aggregated_cash.head()

SK_ID_CURR MONTHS_BALANCE                     CNT_INSTALMENT               \
                       mean     sum   max   min           mean    sum   max   
0   100001.0     -72.555556  -653.0 -53.0 -96.0       4.000000   36.0   4.0   
1   100002.0     -10.000000  -190.0  -1.0 -19.0      24.000000  456.0  24.0   
2   100003.0     -43.785714 -1226.0 -18.0 -77.0      10.107143  283.0  12.0   
3   100004.0     -25.500000  -102.0 -24.0 -27.0       3.750000   15.0   4.0   
4   100005.0     -20.000000  -220.0 -15.0 -25.0      10.545455  116.0  12.0   

        CNT_INSTALMENT_FUTURE  ... NAME_CONTRACT_STATUS_Returned to the store  \
    min                  mean  ...                                        max   
0   4.0              1.444444  ...                                        0.0   
1  24.0             15.000000  ...                                        0.0   
2   6.0              5.785714  ...                                        0.0   
3   3.0              2.250000  ...                                        0.0   
4  -1.0              6.454545  ...                                        0.0   

       NAME_CONTRACT_STATUS_Signed                NAME_CONTRACT_STATUS_XNA  \
   min                        mean  sum  max  min                     mean   
0  0.0                    0.000000  0.0  0.0  0.0                      0.0   
1  0.0                    0.000000  0.0  0.0  0.0                      0.0   
2  0.0                    0.000000  0.0  0.0  0.0                      0.0   
3  0.0                    0.000000  0.0  0.0  0.0                      0.0   
4  0.0                    0.090909  1.0  1.0  0.0                      0.0   

                  
   sum  max  min  
0  0.0  0.0  0.0  
1  0.0  0.0  0.0  
2  0.0  0.0  0.0  
3  0.0  0.0  0.0  
4  0.0  0.0  0.0  

[5 rows x 57 columns]

In [22]:
aggregated_cash.columns = ['_'.join(col).strip('_') for col in aggregated_cash.columns.values]
aggregated_cash

,SK_ID_CURR,MONTHS_BALANCE_mean,MONTHS_BALANCE_sum,MONTHS_BALANCE_max,MONTHS_BALANCE_min,CNT_INSTALMENT_mean,CNT_INSTALMENT_sum,CNT_INSTALMENT_max,CNT_INSTALMENT_min,CNT_INSTALMENT_FUTURE_mean,...,NAME_CONTRACT_STATUS_Returned to the store_max,NAME_CONTRACT_STATUS_Returned to the store_min,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_Signed_sum,NAME_CONTRACT_STATUS_Signed_max,NAME_CONTRACT_STATUS_Signed_min,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_XNA_sum,NAME_CONTRACT_STATUS_XNA_max,NAME_CONTRACT_STATUS_XNA_min
0,100001.0,-72.555556,-653.0,-53.0,-96.0,4.000000,36.0,4.0,4.0,1.444444,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100002.0,-10.000000,-190.0,-1.0,-19.0,24.000000,456.0,24.0,24.0,15.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100003.0,-43.785714,-1226.0,-18.0,-77.0,10.107143,283.0,12.0,6.0,5.785714,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100004.0,-25.500000,-102.0,-24.0,-27.0,3.750000,15.0,4.0,3.0,2.250000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100005.0,-20.000000,-220.0,-15.0,-25.0,10.545455,116.0,12.0,-1.0,6.454545,...,0.0,0.0,0.090909,1.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337247,456251.0,-5.000000,-45.0,-1.0,-9.0,6.888889,62.0,8.0,-1.0,3.777778,...,0.0,0.0,0.111111,1.0,1.0,0.0,0.0,0.0,0.0,0.0
337248,456252.0,-79.000000,-553.0,-76.0,-82.0,6.000000,42.0,6.0,6.0,3.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
337249,456253.0,-79.235294,-1347.0,-57.0,-96.0,6.705882,114.0,12.0,4.0,2.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
337250,456254.0,-5.550000,-111.0,-1.0,-11.0,14.900000,298.0,16.0,14.0,10.350000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Encoding and imputing installments_payments.csv
## Imputing because there are no columns to encode

In [23]:
empty_columns = []

for col in df_instalments.columns:
    if (df_instalments[col].isnull().sum()/10001358 > 0.9):
        empty_columns.append(col)

df_instalments = df_instalments.drop(columns=empty_columns,axis=1)
print(len(empty_columns))
df_instalments.head()

0


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [24]:
imp_mean = SimpleImputer(missing_values = np.nan,strategy='constant',fill_value = -1)
df_instalments = pd.DataFrame(imp_mean.fit_transform(df_instalments),columns=df_instalments.columns)

df_instalments.head()


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186.0,161674.0,1.0,6.0,-1180.0,-1187.0,6948.360,6948.360
1,1330831.0,151639.0,0.0,34.0,-2156.0,-2156.0,1716.525,1716.525
2,2085231.0,193053.0,2.0,1.0,-63.0,-63.0,25425.000,25425.000
3,2452527.0,199697.0,1.0,3.0,-2418.0,-2426.0,24350.130,24350.130
4,2714724.0,167756.0,1.0,2.0,-1383.0,-1366.0,2165.040,2160.585


In [25]:
df_instalments = df_instalments.drop('SK_ID_PREV',axis=1)

In [26]:
aggregated_installments = df_instalments.groupby(['SK_ID_CURR'],as_index=False).agg(['mean','sum','max','min'])
aggregated_installments


SK_ID_CURR NUM_INSTALMENT_VERSION                   \
                                    mean    sum  max  min   
0        100001.0               1.142857    8.0  2.0  1.0   
1        100002.0               1.052632   20.0  2.0  1.0   
2        100003.0               1.040000   26.0  2.0  1.0   
3        100004.0               1.333333    4.0  2.0  1.0   
4        100005.0               1.111111   10.0  2.0  1.0   
...           ...                    ...    ...  ...  ...   
339582   456251.0               1.142857    8.0  2.0  1.0   
339583   456252.0               1.000000    6.0  1.0  1.0   
339584   456253.0               1.000000   14.0  1.0  1.0   
339585   456254.0               1.000000   19.0  1.0  1.0   
339586   456255.0               1.824324  135.0  4.0  1.0   

       NUM_INSTALMENT_NUMBER                   DAYS_INSTALMENT  ...  \
                        mean    sum   max  min            mean  ...   
0                   2.714286   19.0   4.0  1.0    -2187.714286  ...   
1                  10.000000  190.0  19.0  1.0     -295.000000  ...   
2                   5.080000  127.0  12.0  1.0    -1378.160000  ...   
3                   2.000000    6.0   3.0  1.0     -754.000000  ...   
4                   5.000000   45.0   9.0  1.0     -586.000000  ...   
...                      ...    ...   ...  ...             ...  ...   
339582              4.000000   28.0   7.0  1.0     -120.000000  ...   
339583              3.500000   21.0   6.0  1.0    -2391.000000  ...   
339584              4.785714   67.0  12.0  1.0    -2372.928571  ...   
339585              5.263158  100.0  10.0  1.0     -142.263158  ...   
339586              8.851351  655.0  24.0  1.0     -463.945946  ...   

       DAYS_ENTRY_PAYMENT         AMT_INSTALMENT                           \
                      max     min           mean          sum         max   
0                 -1628.0 -2916.0    5885.132143    41195.925   17397.900   
1                   -49.0  -587.0   11559.247105   219625.695   53093.745   
2                  -544.0 -2324.0   64754.586000  1618864.650  560835.360   
3                  -727.0  -795.0    7096.155000    21288.465   10573.965   
4                  -470.0  -736.0    6240.205000    56161.845   17656.245   
...                   ...     ...            ...          ...         ...   
339582              -38.0  -237.0    7492.924286    52450.470   12815.010   
339583            -2327.0 -2470.0   10069.867500    60419.205   10074.465   
339584            -1738.0 -2915.0    4399.707857    61595.910    5575.185   
339585              -18.0  -317.0   10239.832895   194556.825   19065.825   
339586              -76.0  -972.0   41464.713649  3068388.810  615229.515   

                    AMT_PAYMENT                                      
              min          mean          sum         max        min  
0        3951.000   5885.132143    41195.925   17397.900   3951.000  
1        9251.775  11559.247105   219625.695   53093.745   9251.775  
2        6662.970  64754.586000  1618864.650  560835.360   6662.970  
3        5357.250   7096.155000    21288.465   10573.965   5357.250  
4        4813.200   6240.205000    56161.845   17656.245   4813.200  
...           ...           ...          ...         ...        ...  
339582   6605.910   7492.924286    52450.470   12815.010   6605.910  
339583  10046.880  10069.867500    60419.205   10074.465  10046.880  
339584   2754.450   4115.915357    57622.815    5575.185     27.270  
339585   2296.440  10239.832895   194556.825   19065.825   2296.440  
339586  11090.835  47646.215878  3525819.975  669251.655     34.965  

[339587 rows x 25 columns]

In [27]:
aggregated_installments.columns = ['_'.join(col).strip('_') for col in aggregated_installments.columns.values]
aggregated_installments

,SK_ID_CURR,NUM_INSTALMENT_VERSION_mean,NUM_INSTALMENT_VERSION_sum,NUM_INSTALMENT_VERSION_max,NUM_INSTALMENT_VERSION_min,NUM_INSTALMENT_NUMBER_mean,NUM_INSTALMENT_NUMBER_sum,NUM_INSTALMENT_NUMBER_max,NUM_INSTALMENT_NUMBER_min,DAYS_INSTALMENT_mean,...,DAYS_ENTRY_PAYMENT_max,DAYS_ENTRY_PAYMENT_min,AMT_INSTALMENT_mean,AMT_INSTALMENT_sum,AMT_INSTALMENT_max,AMT_INSTALMENT_min,AMT_PAYMENT_mean,AMT_PAYMENT_sum,AMT_PAYMENT_max,AMT_PAYMENT_min
0,100001.0,1.142857,8.0,2.0,1.0,2.714286,19.0,4.0,1.0,-2187.714286,...,-1628.0,-2916.0,5885.132143,41195.925,17397.900,3951.000,5885.132143,41195.925,17397.900,3951.000
1,100002.0,1.052632,20.0,2.0,1.0,10.000000,190.0,19.0,1.0,-295.000000,...,-49.0,-587.0,11559.247105,219625.695,53093.745,9251.775,11559.247105,219625.695,53093.745,9251.775
2,100003.0,1.040000,26.0,2.0,1.0,5.080000,127.0,12.0,1.0,-1378.160000,...,-544.0,-2324.0,64754.586000,1618864.650,560835.360,6662.970,64754.586000,1618864.650,560835.360,6662.970
3,100004.0,1.333333,4.0,2.0,1.0,2.000000,6.0,3.0,1.0,-754.000000,...,-727.0,-795.0,7096.155000,21288.465,10573.965,5357.250,7096.155000,21288.465,10573.965,5357.250
4,100005.0,1.111111,10.0,2.0,1.0,5.000000,45.0,9.0,1.0,-586.000000,...,-470.0,-736.0,6240.205000,56161.845,17656.245,4813.200,6240.205000,56161.845,17656.245,4813.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339582,456251.0,1.142857,8.0,2.0,1.0,4.000000,28.0,7.0,1.0,-120.000000,...,-38.0,-237.0,7492.924286,52450.470,12815.010,6605.910,7492.924286,52450.470,12815.010,6605.910
339583,456252.0,1.000000,6.0,1.0,1.0,3.500000,21.0,6.0,1.0,-2391.000000,...,-2327.0,-2470.0,10069.867500,60419.205,10074.465,10046.880,10069.867500,60419.205,10074.465,10046.880
339584,456253.0,1.000000,14.0,1.0,1.0,4.785714,67.0,12.0,1.0,-2372.928571,...,-1738.0,-2915.0,4399.707857,61595.910,5575.185,2754.450,4115.915357,57622.815,5575.185,27.270
339585,456254.0,1.000000,19.0,1.0,1.0,5.263158,100.0,10.0,1.0,-142.263158,...,-18.0,-317.0,10239.832895,194556.825,19065.825,2296.440,10239.832895,194556.825,19065.825,2296.440


# Encoding and imputing credit_card_balance.csv

In [28]:
columns_card_categorical = df_credit_card.select_dtypes(exclude='number').columns
df_card_categorical = df_credit_card[columns_card_categorical].copy()
column_values_card = df_card_categorical.nunique()

In [29]:
label_card=[]
one_hot_card=[]

for i in range(0,len(column_values_card)):
    if column_values_card.iloc[i]>2:
        one_hot_card.append(column_values_card.index[i])
    else:
        label_card.append(column_values_card.index[i])

print(label_card)
print(one_hot_card)

[]
['NAME_CONTRACT_STATUS']


In [30]:


hot_encoder_card = OneHotEncoder(handle_unknown = 'ignore',sparse_output=False)


hot_card_features = pd.DataFrame(hot_encoder_card.fit_transform(df_credit_card[one_hot_card]),columns=hot_encoder_card.get_feature_names_out(one_hot_card))



In [31]:
hot_card_features

,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
3840307,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3840308,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3840309,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3840310,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
card_numerical_columns = df_credit_card.select_dtypes(include = 'number').columns
df_card_numerical = df_credit_card[card_numerical_columns].copy()
df_card_numerical


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840307,1036507,328243,-9,0.000,45000,NaN,0.0,NaN,NaN,0.000,...,0.000,0.000,0.000,NaN,0,NaN,NaN,0.0,0,0
3840308,1714892,347207,-9,0.000,45000,0.0,0.0,0.0,0.0,0.000,...,0.000,0.000,0.000,0.0,0,0.0,0.0,23.0,0,0
3840309,1302323,215757,-9,275784.975,585000,270000.0,270000.0,0.0,0.0,2250.000,...,269356.140,273093.975,273093.975,2.0,2,0.0,0.0,18.0,0,0
3840310,1624872,430337,-10,0.000,450000,NaN,0.0,NaN,NaN,0.000,...,0.000,0.000,0.000,NaN,0,NaN,NaN,0.0,0,0


In [33]:
df_credit_card = pd.concat([df_card_numerical,hot_card_features],axis=1)
df_credit_card

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,35.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,69.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,30.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,10.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,101.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840307,1036507,328243,-9,0.000,45000,NaN,0.0,NaN,NaN,0.000,...,0.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3840308,1714892,347207,-9,0.000,45000,0.0,0.0,0.0,0.0,0.000,...,23.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3840309,1302323,215757,-9,275784.975,585000,270000.0,270000.0,0.0,0.0,2250.000,...,18.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3840310,1624872,430337,-10,0.000,450000,NaN,0.0,NaN,NaN,0.000,...,0.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
empty_columns = []

for col in df_credit_card.columns:
    if (df_credit_card[col].isnull().sum()/3840312 > 0.9):
        empty_columns.append(col)

df_credit_card = df_credit_card.drop(columns=empty_columns, axis=1)
print(len(empty_columns))
df_credit_card.head()


0


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,35.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,69.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,30.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,10.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,101.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
imp_mean = SimpleImputer(missing_values=np.nan,strategy='constant',fill_value=-1)
df_credit_card = pd.DataFrame(imp_mean.fit_transform(df_credit_card),columns=df_credit_card.columns)

df_credit_card.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed
0,2562384.0,378907.0,-6.0,56.970,135000.0,0.0,877.5,0.0,877.5,1700.325,...,35.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2582071.0,363914.0,-1.0,63975.555,45000.0,2250.0,2250.0,0.0,0.0,2250.000,...,69.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1740877.0,371185.0,-7.0,31815.225,450000.0,0.0,0.0,0.0,0.0,2250.000,...,30.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1389973.0,337855.0,-4.0,236572.110,225000.0,2250.0,2250.0,0.0,0.0,11795.760,...,10.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1891521.0,126868.0,-1.0,453919.455,450000.0,0.0,11547.0,0.0,11547.0,22924.890,...,101.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
df_credit_card = df_credit_card.drop('SK_ID_PREV',axis=1)

In [37]:
aggregated_credit_card = df_credit_card.groupby(['SK_ID_CURR'],as_index=False).agg(['mean','sum','max','min']
    
)

aggregated_credit_card

SK_ID_CURR MONTHS_BALANCE                       AMT_BALANCE  \
                            mean     sum  max   min           mean   
0        100006.0           -3.5   -21.0 -1.0  -6.0       0.000000   
1        100011.0          -38.5 -2849.0 -2.0 -75.0   54482.111149   
2        100013.0          -48.5 -4656.0 -1.0 -96.0   18159.919219   
3        100021.0          -10.0  -170.0 -2.0 -18.0       0.000000   
4        100023.0           -7.5   -60.0 -4.0 -11.0       0.000000   
...           ...            ...     ...  ...   ...            ...   
103553   456244.0          -21.0  -861.0 -1.0 -41.0  131834.730732   
103554   456246.0           -5.5   -44.0 -2.0  -9.0   13136.731875   
103555   456247.0          -49.0 -4655.0 -2.0 -96.0   23216.396211   
103556   456248.0          -13.0  -299.0 -2.0 -24.0       0.000000   
103557   456250.0           -6.5   -78.0 -1.0 -12.0  173589.326250   

                                            AMT_CREDIT_LIMIT_ACTUAL  ...  \
                sum         max         min                    mean  ...   
0             0.000       0.000       0.000           270000.000000  ...   
1       4031676.225  189000.000       0.000           164189.189189  ...   
2       1743352.245  161420.220       0.000           131718.750000  ...   
3             0.000       0.000       0.000           675000.000000  ...   
4             0.000       0.000       0.000           135000.000000  ...   
...             ...         ...         ...                     ...  ...   
103553  5405223.960  453627.675       0.000           296341.463415  ...   
103554   105093.855   43490.115       0.000           135000.000000  ...   
103555  2205557.640  190202.130       0.000           144000.000000  ...   
103556        0.000       0.000       0.000           900000.000000  ...   
103557  2083071.915  200208.915  153832.725           178875.000000  ...   

       NAME_CONTRACT_STATUS_Refused      NAME_CONTRACT_STATUS_Sent proposal  \
                                max  min                               mean   
0                               0.0  0.0                                0.0   
1                               0.0  0.0                                0.0   
2                               0.0  0.0                                0.0   
3                               0.0  0.0                                0.0   
4                               0.0  0.0                                0.0   
...                             ...  ...                                ...   
103553                          0.0  0.0                                0.0   
103554                          0.0  0.0                                0.0   
103555                          0.0  0.0                                0.0   
103556                          0.0  0.0                                0.0   
103557                          0.0  0.0                                0.0   

                      NAME_CONTRACT_STATUS_Signed                 
        sum  max  min                        mean  sum  max  min  
0       0.0  0.0  0.0                         0.0  0.0  0.0  0.0  
1       0.0  0.0  0.0                         0.0  0.0  0.0  0.0  
2       0.0  0.0  0.0                         0.0  0.0  0.0  0.0  
3       0.0  0.0  0.0                         0.0  0.0  0.0  0.0  
4       0.0  0.0  0.0                         0.0  0.0  0.0  0.0  
...     ...  ...  ...                         ...  ...  ...  ...  
103553  0.0  0.0  0.0                         0.0  0.0  0.0  0.0  
103554  0.0  0.0  0.0                         0.0  0.0  0.0  0.0  
103555  0.0  0.0  0.0                         0.0  0.0  0.0  0.0  
103556  0.0  0.0  0.0                         0.0  0.0  0.0  0.0  
103557  0.0  0.0  0.0                         0.0  0.0  0.0  0.0  

[103558 rows x 109 columns]

In [38]:
aggregated_credit_card.columns = ['_'.join(col).strip('_') for col in aggregated_credit_card.columns.values]
aggregated_credit_card

,SK_ID_CURR,MONTHS_BALANCE_mean,MONTHS_BALANCE_sum,MONTHS_BALANCE_max,MONTHS_BALANCE_min,AMT_BALANCE_mean,AMT_BALANCE_sum,AMT_BALANCE_max,AMT_BALANCE_min,AMT_CREDIT_LIMIT_ACTUAL_mean,...,NAME_CONTRACT_STATUS_Refused_max,NAME_CONTRACT_STATUS_Refused_min,NAME_CONTRACT_STATUS_Sent proposal_mean,NAME_CONTRACT_STATUS_Sent proposal_sum,NAME_CONTRACT_STATUS_Sent proposal_max,NAME_CONTRACT_STATUS_Sent proposal_min,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_Signed_sum,NAME_CONTRACT_STATUS_Signed_max,NAME_CONTRACT_STATUS_Signed_min
0,100006.0,-3.5,-21.0,-1.0,-6.0,0.000000,0.000,0.000,0.000,270000.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100011.0,-38.5,-2849.0,-2.0,-75.0,54482.111149,4031676.225,189000.000,0.000,164189.189189,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100013.0,-48.5,-4656.0,-1.0,-96.0,18159.919219,1743352.245,161420.220,0.000,131718.750000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100021.0,-10.0,-170.0,-2.0,-18.0,0.000000,0.000,0.000,0.000,675000.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100023.0,-7.5,-60.0,-4.0,-11.0,0.000000,0.000,0.000,0.000,135000.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103553,456244.0,-21.0,-861.0,-1.0,-41.0,131834.730732,5405223.960,453627.675,0.000,296341.463415,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103554,456246.0,-5.5,-44.0,-2.0,-9.0,13136.731875,105093.855,43490.115,0.000,135000.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103555,456247.0,-49.0,-4655.0,-2.0,-96.0,23216.396211,2205557.640,190202.130,0.000,144000.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103556,456248.0,-13.0,-299.0,-2.0,-24.0,0.000000,0.000,0.000,0.000,900000.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Merging all dataframes on SK_ID_CURR

In [39]:
print(temp_aggregated.shape)
print(aggregated_cash.shape)
print(aggregated_installments.shape)
print(aggregated_credit_card.shape)

(338857, 661)
(337252, 57)
(339587, 25)
(103558, 109)


In [40]:
def force_optimize_all_to_int(df):
    for col in df.columns:
        col_data = df[col]

        # Skip non-numeric columns
        if not pd.api.types.is_numeric_dtype(col_data):
            continue

        # If float → round and convert to int64 first
        if pd.api.types.is_float_dtype(col_data):
            col_data = col_data.round().astype('int64')

        # Handle integer downcasting
        col_min = col_data.min()
        col_max = col_data.max()

        if col_min >= -128 and col_max <= 127:
            df[col] = col_data.astype('int8')
        elif col_min >= -32_768 and col_max <= 32_767:
            df[col] = col_data.astype('int16')
        elif col_min >= -2_147_483_648 and col_max <= 2_147_483_647:
            df[col] = col_data.astype('int32')
        else:
            df[col] = col_data.astype('int64')  # leave as is if very large

    return df


In [41]:
print("Before:")
print(temp_aggregated.memory_usage(deep=True).sum() / 1024**2, "MB")

temp_aggregated = force_optimize_all_to_int(temp_aggregated)

print("After:")
print(temp_aggregated.memory_usage(deep=True).sum() / 1024**2, "MB")


Before:
1708.8660697937012 MB
After:
270.4843912124634 MB


In [42]:
aggregated_cash = force_optimize_all_to_int(aggregated_cash)

In [43]:
aggregated_installments = force_optimize_all_to_int(aggregated_installments)

In [44]:
aggregated_credit_card = force_optimize_all_to_int(aggregated_credit_card)

In [45]:
df_merged = pd.merge(temp_aggregated,aggregated_installments,on='SK_ID_CURR',how='left')

In [46]:
df_merged2 = pd.merge(aggregated_cash,aggregated_credit_card,on='SK_ID_CURR',how='left')

In [47]:
df = pd.merge(df_merged,df_merged2,on='SK_ID_CURR',how='left')
df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,SK_ID_CURR,AMT_ANNUITY_mean,AMT_ANNUITY_sum,AMT_ANNUITY_max,AMT_ANNUITY_min,AMT_APPLICATION_mean,AMT_APPLICATION_sum,AMT_APPLICATION_max,AMT_APPLICATION_min,AMT_CREDIT_mean,...,NAME_CONTRACT_STATUS_Refused_max_y,NAME_CONTRACT_STATUS_Refused_min_y,NAME_CONTRACT_STATUS_Sent proposal_mean,NAME_CONTRACT_STATUS_Sent proposal_sum,NAME_CONTRACT_STATUS_Sent proposal_max,NAME_CONTRACT_STATUS_Sent proposal_min,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_Signed_sum_y,NAME_CONTRACT_STATUS_Signed_max_y,NAME_CONTRACT_STATUS_Signed_min_y
0,100001,3951,3951,3951,3951,24836,24836,24836,24836,23787,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100002,9252,9252,9252,9252,179055,179055,179055,179055,179055,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100003,56554,169662,98357,6737,435436,1306310,900000,68810,484191,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100004,5357,5357,5357,5357,24282,24282,24282,24282,20106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100005,2406,4812,4813,-1,22309,44618,44618,0,20077,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338852,456251,6606,6606,6606,6606,40455,40455,40455,40455,40455,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338853,456252,10074,10074,10074,10074,57596,57596,57596,57596,56822,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338854,456253,4770,9541,5568,3973,24163,48326,28912,19413,20626,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338855,456254,10681,21362,19066,2296,121318,242636,223790,18846,134440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
def impute_in_chunks(df, chunk_size=100):
    imputer = SimpleImputer(strategy='constant',fill_value=-1)
    chunks = []

    for i in range(0, df.shape[1], chunk_size):
        chunk = df.iloc[:, i:i+chunk_size]
        chunk_imputed = pd.DataFrame(imputer.fit_transform(chunk), columns=chunk.columns)
        chunks.append(chunk_imputed)

    return pd.concat(chunks, axis=1)

In [53]:
df = impute_in_chunks(df)
df

,SK_ID_CURR,AMT_ANNUITY_mean,AMT_ANNUITY_sum,AMT_ANNUITY_max,AMT_ANNUITY_min,AMT_APPLICATION_mean,AMT_APPLICATION_sum,AMT_APPLICATION_max,AMT_APPLICATION_min,AMT_CREDIT_mean,...,NAME_CONTRACT_STATUS_Refused_max_y,NAME_CONTRACT_STATUS_Refused_min_y,NAME_CONTRACT_STATUS_Sent proposal_mean,NAME_CONTRACT_STATUS_Sent proposal_sum,NAME_CONTRACT_STATUS_Sent proposal_max,NAME_CONTRACT_STATUS_Sent proposal_min,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_Signed_sum_y,NAME_CONTRACT_STATUS_Signed_max_y,NAME_CONTRACT_STATUS_Signed_min_y
0,100001,3951,3951,3951,3951,24836,24836,24836,24836,23787,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,100002,9252,9252,9252,9252,179055,179055,179055,179055,179055,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,100003,56554,169662,98357,6737,435436,1306310,900000,68810,484191,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,100004,5357,5357,5357,5357,24282,24282,24282,24282,20106,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,100005,2406,4812,4813,-1,22309,44618,44618,0,20077,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338852,456251,6606,6606,6606,6606,40455,40455,40455,40455,40455,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
338853,456252,10074,10074,10074,10074,57596,57596,57596,57596,56822,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
338854,456253,4770,9541,5568,3973,24163,48326,28912,19413,20626,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
338855,456254,10681,21362,19066,2296,121318,242636,223790,18846,134440,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [54]:
from pathlib import Path  
filepath = Path('/kaggle/working/previous_loans_int.csv')

df.to_csv(filepath, index=False)
